# Exercise 1: time-advancing scheme and neural networks

Exercise on the implementation of Neural ODEs. 

Author: Stefano Pagani <stefano.pagani@polimi.it>.

Date: 2024

Course: Mathematical and numerical foundations of scientific machine learning.




Let us consider the Lorenz system of differential equations:

$
\dot{x} = \sigma (y-x) \\
\dot{y} = x (\rho-z) - y \\
\dot{z} = x y - \beta z
$

with parameters $\sigma = 10$, $\rho = 28$, and $\beta = 8/3$.

The goal of this exercise is to implement a feed-forward neural network that learns the right-hand side of a dynamical system, which allows to approximate the dynamics of the state.

Complete the notebook accordingly.

In [ ]:

# imports

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K



In [ ]:


rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [9, 9]

## Simulate the Lorenz System for various initial conditions

dt = 0.01
T = 1.0
t = np.arange(0,T+dt,dt)
beta = 8/3
sigma = 10
rho = 28

n_samples = 10
n_valid = 5

nn_input = np.zeros((n_samples*(len(t)-1),3))
nn_output = np.zeros_like(nn_input)

nn_input_val = np.zeros((n_valid*(len(t)-1),3))
nn_output_val = np.zeros_like(nn_input_val)

nn_pred = np.zeros_like(nn_input_val)

def lorenz_deriv(t0, x_y_z, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

np.random.seed(123)
x0 = 1.5 + 3.0 * np.random.random((n_samples, 3))
x0_val = 1.5 + 3.0 * np.random.random((n_valid, 3))

x_t = np.asarray([integrate.solve_ivp(lorenz_deriv, t_span=(0,T), y0=x0_j, t_eval=t).y for x0_j in x0])

x_t_val = np.asarray([integrate.solve_ivp(lorenz_deriv, (0,T), y0=x0_j, t_eval=t).y for x0_j in x0_val])



In [ ]:
fig,ax = plt.subplots(1,1,subplot_kw={'projection': '3d'})

# training data
for j in range(n_samples):

    nn_input[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t[j,:,:-1].T
    nn_output[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t[j,:,1:].T

    x, y, z = x_t[j,:,:]
    ax.plot(x, y, z,linewidth=1)
    ax.scatter(x0[j,0],x0[j,1],x0[j,2],color='r')

ax.view_init(18, -113)
plt.show()

# test data
for j in range(n_valid):
    nn_input_val[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t_val[j,:,:-1].T
    nn_output_val[j*(len(t)-1):(j+1)*(len(t)-1),:] = x_t_val[j,:,1:].T



In [ ]:
# convert to tensor

X_train = tf.convert_to_tensor(x_t[:,:,:-1],dtype=tf.float64)
Y_train = tf.convert_to_tensor(x_t[:,:,1:],dtype=tf.float64)

X_valid = tf.convert_to_tensor(x_t_val[:,:,:-1],dtype=tf.float64)
Y_valid = tf.convert_to_tensor(x_t_val[:,:,1:],dtype=tf.float64)

Y_pred = tf.convert_to_tensor(nn_pred,dtype=tf.float64)


In [ ]:
#  loss function
def loss(X_train,Y_train):

    # single time step loss
    for k in range(100):
        if k==0:
            Y_pred = tf.expand_dims(Updatemodel(X_train[:,:,0]),2)
        else:
            Y_pred = tf.concat( [Y_pred, tf.expand_dims(Updatemodel(X_train[:,:,k]),2) ], 2 )

    # loss components
    mse_loss = tf.reduce_mean(tf.pow(Y_pred-Y_train,2))

    return mse_loss

# neural network weight gradients
@tf.function
def grad(model,X_train,Y_train):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(X_train,Y_train)
        grads = tape.gradient(loss_value,model.trainable_variables)
    return loss_value, grads






Task: Compare the following models:

  1. $\bf{x}^{(k+1)} = \mathcal{NN}(\bf{x}^{(k)}; \bf{W})  $
  2. $\bf{x}^{(k+1)} = \bf{x}^{(k)} + \Delta t \mathcal{NN}(\bf{x}^{(k)}; \bf{W}) $


In [ ]:

dense = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation=None, input_shape=(3,),kernel_initializer="glorot_normal",dtype=tf.float64),
    tf.keras.layers.Dense(16, activation='tanh',kernel_initializer="glorot_normal",dtype=tf.float64),
    tf.keras.layers.Dense(3,activation=None,kernel_initializer="glorot_normal",dtype=tf.float64)
])

# inputs      = tf.keras.Input(shape=(3,))
# outputs     = dense(inputs)
# Updatemodel = tf.keras.Model(inputs=inputs, outputs=outputs)

inputs      = tf.keras.Input(shape=(3,),dtype=tf.float64)
outputs     = inputs[:,0:3] + 0.01*dense(inputs)
Updatemodel = tf.keras.Model(inputs=inputs, outputs=outputs)




In [ ]:
# Adam optimizer
tf_optimizer = tf.keras.optimizers.Adam(learning_rate=0.003,beta_1=0.99)

for iter in range(10000):

  # compute gradients using AD
  loss_value,grads = grad(Updatemodel,X_train,Y_train)

  # update neural network weights
  tf_optimizer.apply_gradients(zip(grads,Updatemodel.trainable_variables))

  # display intermediate results
  if ((iter+1) % 200 == 0):
    print('iter =  '+str(iter+1))
    print('loss = {:.16f}'.format(loss_value))
    print('loss_valid = {:.16f}'.format(loss(X_valid,Y_valid)))
   

In [ ]:


## NN prediction on the test set
fig,ax = plt.subplots(1,1,subplot_kw={'projection': '3d'})

for k in range(100):
    if k==0:
        Y_plot = tf.expand_dims(Updatemodel(X_valid[:,:,0]),2)
    else:
        Y_plot = tf.concat( [Y_plot, tf.expand_dims(Updatemodel(Y_plot[:,:,-1]),2) ], 2 )

j=1

x = Y_valid.numpy()[j,0,:]
y = Y_valid.numpy()[j,1,:]
z = Y_valid.numpy()[j,2,:]
ax.plot(x, y, z,linewidth=1)
x = Y_plot.numpy()[j,0,:]
y = Y_plot.numpy()[j,1,:]
z = Y_plot.numpy()[j,2,:]
ax.plot(x, y, z,'--',linewidth=1)
    
ax.view_init(18, -113)
plt.show()




Task: implement a custum loss to perform a further training of the neural network. 

In [ ]:
def loss(X_train,Y_train):
    
    #Y_pred = Updatemodel(X_train)

    # custom loss
    # ....

    # loss components
    # mse_loss = ...

    return mse_loss

# neural network weight gradients
@tf.function
def grad(model,X_train,Y_train):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(X_train,Y_train)
        grads = tape.gradient(loss_value,model.trainable_variables)
    return loss_value, grads


In [ ]:
# Adam optimizer
tf_optimizer = tf.keras.optimizers.Adam(learning_rate=0.003,beta_1=0.99)

for iter in range(10000):

  # compute gradients using AD
  loss_value,grads = grad(Updatemodel,X_train,Y_train)

  # update neural network weights
  tf_optimizer.apply_gradients(zip(grads,Updatemodel.trainable_variables))

  # display intermediate results
  if ((iter+1) % 200 == 0):
    print('iter =  '+str(iter+1))
    print('loss = {:.16f}'.format(loss_value))
    print('loss_valid = {:.16f}'.format(loss(X_valid,Y_valid)))

In [ ]:
## NN prediction on the test set
fig,ax = plt.subplots(1,1,subplot_kw={'projection': '3d'})

for k in range(100):
    if k==0:
        Y_plot = tf.expand_dims(Updatemodel(X_valid[:,:,0]),2)
    else:
        Y_plot = tf.concat( [Y_plot, tf.expand_dims(Updatemodel(Y_plot[:,:,-1]),2) ], 2 )

j=0

x = Y_valid.numpy()[j,0,:]
y = Y_valid.numpy()[j,1,:]
z = Y_valid.numpy()[j,2,:]
ax.plot(x, y, z,linewidth=1)
x = Y_plot.numpy()[j,0,:]
y = Y_plot.numpy()[j,1,:]
z = Y_plot.numpy()[j,2,:]
ax.plot(x, y, z,'--',linewidth=1)
    
ax.view_init(18, -113)
plt.show()

Task: train using different configurations (and optmization settings)